# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat publications.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

C:\Users\26595\AppData\Local\Temp\ipykernel_14552\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [5]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2022-09-01,Bayesian optimization of nanophotonic electrom...,Optics Express,Optimizing nanophotonic electromagnetic shield...,"Li, Mingxuan; McCourt, Michael J; Galante, Ant...",bayesian-optimization-nanophotonic,https://www.lamp.pitt.edu/Publications/056.pdf
1,2022-07-01,"Achieving highly conductive, stretchable, and ...",ACS Applied Polymer Materials,"Method for creating highly conductive, stretch...","Galante, Anthony J; Pilsbury, Brady C; Li, Min...",highly-conductive-stretchable-fabric,https://www.lamp.pitt.edu/Publications/054.pdf
2,2022-01-01,"Ink-coated silver films on PET for flexible, h...",ACS Applied Electronic Materials,Using ink-coated silver films on PET substrate...,"Li, Mingxuan; Sinha, Sneh; Hannani, Sima; Walk...",ink-coated-silver-films,https://www.lamp.pitt.edu/Publications/058.pdf
3,2023-01-01,Stretchable and wash durable reactive silver i...,Progress in Organic Coatings,"Stretchable, wash durable silver ink coatings ...","Li, Mingxuan; Zarei, Mehdi; Galante, Anthony J...",stretchable-wash-durable-coatings,https://www.lamp.pitt.edu/Publications/059.pdf
4,2023-01-01,Silver meshes for record-performance transpare...,ACS Applied Materials & Interfaces,Silver meshes achieving record performance in ...,"Li, Mingxuan; Zarei, Mehdi; Mohammadi, Khashay...",silver-meshes-record-performance,https://www.lamp.pitt.edu/Publications/060.pdf
5,2023-10-01,Substrate-embedded metal meshes for ITO-free o...,Optics Express,Metal meshes embedded in substrates for ITO-fr...,"Zarei, Mehdi; Loy, James C; Li, Mingxuan; Zhou...",substrate-embedded-metal-meshes,https://www.lamp.pitt.edu/Publications/061.pdf
6,2024-01-01,Datasets and Benchmarks for Nanophotonic Struc...,Advances in Neural Information Processing Systems,Introducing datasets and benchmarks for simula...,"Kim, Jungtaek; Li, Mingxuan; Hinder, Oliver; L...",datasets-benchmarks-nanophotonic,https://www.lamp.pitt.edu/Publications/063.pdf
7,2024-02-01,Multi-BOWS: multi-fidelity multi-objective Bay...,Digital Discovery,Multi-fidelity and multi-objective Bayesian op...,"Kim, Jungtaek; Li, Mingxuan; Li, Yirong; Gómez...",multi-bows-nanophotonic-design,https://www.lamp.pitt.edu/Publications/064.pdf
8,2024-03-01,Flexible Embedded Metal Meshes by Sputter-Free...,ACS Applied Materials & Interfaces,Flexible metal meshes created by sputter-free ...,"Zarei, Mehdi; Li, Mingxuan; Medvedeva, Elizabe...",flex-metal-meshes-crack-lithography,https://www.lamp.pitt.edu/Publications/065.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [6]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [7]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
!ls ../_publications/

In [8]:
!cat ../_publications/2009-10-01-paper-title-number-1.md